In [5]:
using Distributed
using LaTeXStrings
using Plots

In [6]:
include("./mps.jl");

In [7]:
include("./mpo.jl");

LoadError: LoadError: [91msyntax: extra token "as" after end of expression[39m
in expression starting at /home/vvitale/Tensor-Networks/mpo.jl:6

In [8]:
include("./dmrg.jl");

LoadError: LoadError: LoadError: [91msyntax: extra token "as" after end of expression[39m
in expression starting at /home/vvitale/Tensor-Networks/mpo.jl:6
in expression starting at /home/vvitale/Tensor-Networks/dmrg.jl:6

# Test DMRG

In [5]:
cols=3
rows=3
config=zeros(Int,rows,cols)
k=0
for j in 1:cols
    for i in 1:rows
        k+=1
        config[i,j]=k
    end
end
config

3×3 Matrix{Int64}:
 1  4  7
 2  5  8
 3  6  9

In [6]:
d=2
chi=10
chimax=64
sweeps=10
J1=1.
J2=0.5
M=MPO()
Initialize!("J1-J2_ITensors",M,J1,J2,cols,rows,config);

In [7]:
ψ=MPS()
Initialize!(ψ,d,chi,cols*rows)
two_sites_dmrg!(ψ,M,sweeps,chimax)

Done! Energy= -15.837359989589917; Variance: -5.684341886080802e-14


# Test RUC

In [ ]:
include("./RUC.jl")